In [2]:
import pandas as pd
from pathlib import Path
from scipy.stats import ttest_ind, mannwhitneyu

In [3]:
FILE_PREFIXES = {
    "cbench": "cbench_stat_results_on_",
    "mibench": "mibench_stat_results_on_",
}

In [4]:
def analyze_final_results():
    """
    Загружает данные с обеих платформ, проводит статистические тесты
    и выводит итоговые результаты по переносимости.
    """
    for suite_name, prefix in FILE_PREFIXES.items():
        print(f"\n{'='*25} АНАЛИЗ НАБОРА: {suite_name.upper()} {'='*25}")
        
        try:
            df_on_x86 = pd.read_csv(f"{prefix}x86_64.csv")
            df_on_arm = pd.read_csv(f"{prefix}arm64.csv")
        except FileNotFoundError as e:
            print(f"ПРЕДУПРЕЖДЕНИЕ: Не найден файл результатов: {e}. Пропускаем набор {suite_name}.")
            continue

        common_benchmarks = sorted(list(
            set(df_on_x86['benchmark'].unique()) & set(df_on_arm['benchmark'].unique())
        ))

        significant_non_transferable = 0

        for benchmark_name in common_benchmarks:
            print(f"\n--- Бенчмарк: {benchmark_name} ---")

            # --- 1. Анализ на платформе x86_64 ---
            print("  Анализ на платформе x86_64:")
            
            native_runs = df_on_x86[(df_on_x86['benchmark'] == benchmark_name) & (df_on_x86['flags_source_arch'] == 'x86_64')]['exec_time']
            foreign_runs = df_on_x86[(df_on_x86['benchmark'] == benchmark_name) & (df_on_x86['flags_source_arch'] == 'arm64')]['exec_time']

            if native_runs.empty or foreign_runs.empty or native_runs.isnull().all() or foreign_runs.isnull().all():
                print("    Недостаточно данных для сравнения на x86_64.")
            else:
                # ### ИЗМЕНЕНИЕ: Проводим и выводим оба теста ###
                t_stat, t_pvalue = ttest_ind(native_runs, foreign_runs, equal_var=False, alternative='less', nan_policy='omit')
                u_stat, u_pvalue = mannwhitneyu(native_runs, foreign_runs, alternative='less', nan_policy='omit')

                print(f"    - Медиана со 'своими' флагами (x86): {native_runs.median():.6f} сек.")
                print(f"    - Медиана с 'чужими' флагами (arm): {foreign_runs.median():.6f} сек.")
                print(f"    - p-value (t-тест):              {t_pvalue:.4f}")
                print(f"    - p-value (Тест Манна-Уитни):    {u_pvalue:.4f}")

                if u_pvalue < 0.05:
                    print("    -> ВЫВОД (по Манну-Уитни): 'Свои' флаги СТАТИСТИЧЕСКИ ЗНАЧИМО лучше. Переносимость плохая.")
                    significant_non_transferable += 1
                else:
                    print("    -> ВЫВОД (по Манну-Уитни): Нет стат. значимых доказательств, что 'свои' флаги лучше. Переносимость хорошая.")

            # --- 2. Анализ на платформе arm64 ---
            print("  Анализ на платформе arm64:")
            
            native_runs = df_on_arm[(df_on_arm['benchmark'] == benchmark_name) & (df_on_arm['flags_source_arch'] == 'arm64')]['exec_time']
            foreign_runs = df_on_arm[(df_on_arm['benchmark'] == benchmark_name) & (df_on_arm['flags_source_arch'] == 'x86_64')]['exec_time']

            if native_runs.empty or foreign_runs.empty or native_runs.isnull().all() or foreign_runs.isnull().all():
                print("    Недостаточно данных для сравнения на arm64.")
            else:
                # ### ИЗМЕНЕНИЕ: Проводим и выводим оба теста ###
                t_stat, t_pvalue = ttest_ind(native_runs, foreign_runs, equal_var=False, alternative='less', nan_policy='omit')
                u_stat, u_pvalue = mannwhitneyu(native_runs, foreign_runs, alternative='less', nan_policy='omit')

                print(f"    - Медиана со 'своими' флагами (arm): {native_runs.median():.6f} сек.")
                print(f"    - Медиана с 'чужими' флагами (x86): {foreign_runs.median():.6f} сек.")
                print(f"    - p-value (t-тест):              {t_pvalue:.4f}")
                print(f"    - p-value (Тест Манна-Уитни):    {u_pvalue:.4f}")

                if u_pvalue < 0.05:
                    print("    -> ВЫВОД (по Манну-Уитни): 'Свои' флаги СТАТИСТИЧЕСКИ ЗНАЧИМО лучше. Переносимость плохая.")
                    significant_non_transferable += 1
                else:
                    print("    -> ВЫВОД (по Манну-Уитни): Нет стат. значимых доказательств, что 'свои' флаги лучше. Переносимость хорошая.")
        
        print(f"\n--- Итоги по набору {suite_name.upper()} ---")
        total_comparisons = len(common_benchmarks) * 2
        print(f"Всего проведено сравнений: {total_comparisons}")
        print(f"Случаев со статистически значимой непереносимостью (p < 0.05 по Манну-Уитни): {significant_non_transferable}")
        if total_comparisons > 0:
            percentage = (significant_non_transferable / total_comparisons) * 100
            print(f"Процент непереносимых случаев: {percentage:.1f}%")


In [5]:
analyze_final_results()


========================= АНАЛИЗ НАБОРА: CBENCH =========================

--- Бенчмарк: cat-cat1 ---
  Анализ на платформе x86_64:
    - Медиана со 'своими' флагами (x86): 0.049705 сек.
    - Медиана с 'чужими' флагами (arm): 0.049493 сек.
    - p-value (t-тест):              0.9980
    - p-value (Тест Манна-Уитни):    1.0000
    -> ВЫВОД (по Манну-Уитни): Нет стат. значимых доказательств, что 'свои' флаги лучше. Переносимость хорошая.
  Анализ на платформе arm64:
    - Медиана со 'своими' флагами (arm): 0.280418 сек.
    - Медиана с 'чужими' флагами (x86): 0.280127 сек.
    - p-value (t-тест):              0.7965
    - p-value (Тест Манна-Уитни):    1.0000
    -> ВЫВОД (по Манну-Уитни): Нет стат. значимых доказательств, что 'свои' флаги лучше. Переносимость хорошая.

--- Бенчмарк: cat-cat2 ---
  Анализ на платформе x86_64:
    - Медиана со 'своими' флагами (x86): 0.002042 сек.
    - Медиана с 'чужими' флагами (arm): 0.001778 сек.
    - p-value (t-тест):              1.0000
    - p-v